In [86]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

def q_change(se):
    return((se.iloc[-1] - se.iloc[0])/se.iloc[0])

pd.options.display.max_columns = 999

In [2]:
## get stock data
companies = ["Walmart", "Amazon", "Costco", "Target", "Kroger"]

stock = pd.DataFrame()
for company in companies:
    
    df = pd.read_csv(company+".csv", parse_dates=["Date"])
    df["quarter"] = pd.PeriodIndex(df.Date, freq="Q")
    df["Price"] = df["Price"].map(lambda x: float(str(x).replace(',','')))
    df = df.sort_values(by=["Date"])
    tmp = df.groupby(by=["quarter"])[["Price"]].agg(q_change)
    tmp = tmp.reset_index()
    tmp["company"] = company
    stock = pd.concat([stock, tmp], ignore_index=True)

stock["quarter"] = stock["quarter"].map(lambda x: str(x))

#### Build S&P 500 index

In [92]:
df = pd.read_csv("SP500.csv", parse_dates=["Date"])
df["quarter"] = pd.PeriodIndex(df.Date, freq="Q")
df["Price"] = df["Price"].map(lambda x: float(str(x).replace(',','')))
df = df.sort_values(by=["Date"])
tmp = df.groupby(by=["quarter"])[["Price"]].agg(q_change)
tmp = tmp.reset_index()
sp = tmp.copy()
sp["quarter"] = sp["quarter"].map(lambda x: str(x))
sp.columns = ["quarter", "sp_index_change"]
sp.head()

,quarter,sp_index_change
0,2010Q1,0.032163
1,2010Q2,-0.125108
2,2010Q3,0.110797
3,2010Q4,0.097187
4,2011Q1,0.042426


In [97]:
topic_revenue = pd.read_csv("topic_revenue.csv")
topic_revenue.shape

(189, 19)

In [98]:
## get revenue data
topic_revenue = pd.read_csv("topic_revenue.csv")
revenue = pd.read_csv("revenue.csv")
revenue = revenue[["actual_quarter", "company", "yoy_growth"]]
topic_revenue = topic_revenue.merge(revenue, left_on=["company", "yoy_growth"], right_on = ["company", "yoy_growth"])
topic_revenue.head()

,1,2,3,4,5,6,7,8,9,10,11,12,company,Revenue,yoy_growth,pci,Q2,Q3,Q4,actual_quarter
0,0.027778,0.083333,0.305556,0.000000,0.027778,0.194444,0.111111,0.083333,0.166667,0.000000,0.000000,0.00000,Walmart,99.81,0.059104,0.401415,0,0,0,2010Q1
1,0.000000,0.076923,0.256410,0.051282,0.051282,0.179487,0.025641,0.102564,0.179487,0.025641,0.051282,0.00000,Walmart,103.73,0.028251,0.475072,1,0,0,2010Q2
2,0.023810,0.095238,0.285714,0.095238,0.023810,0.166667,0.000000,0.119048,0.142857,0.023810,0.000000,0.02381,Walmart,101.95,0.025964,0.093250,0,1,0,2010Q3
3,0.000000,0.051282,0.333333,0.153846,0.000000,0.230769,0.000000,0.076923,0.128205,0.000000,0.025641,0.00000,Walmart,116.36,0.024115,0.294916,0,0,1,2010Q4
4,0.000000,0.071429,0.309524,0.047619,0.000000,0.261905,0.023810,0.119048,0.119048,0.000000,0.023810,0.02381,Walmart,104.19,0.043883,1.264822,0,0,0,2011Q1


In [115]:
topic_stock = topic_revenue.merge(stock, left_on=["company", "actual_quarter"], right_on=["company", "quarter"])
topic_stock.shape
topic_stock.head()

,1,2,3,4,5,6,7,8,9,10,11,12,company,Revenue,yoy_growth,pci,Q2,Q3,Q4,actual_quarter,quarter,Price
0,0.027778,0.083333,0.305556,0.000000,0.027778,0.194444,0.111111,0.083333,0.166667,0.000000,0.000000,0.00000,Walmart,99.81,0.059104,0.401415,0,0,0,2010Q1,2010Q1,0.025263
1,0.000000,0.076923,0.256410,0.051282,0.051282,0.179487,0.025641,0.102564,0.179487,0.025641,0.051282,0.00000,Walmart,103.73,0.028251,0.475072,1,0,0,2010Q2,2010Q2,-0.133718
2,0.023810,0.095238,0.285714,0.095238,0.023810,0.166667,0.000000,0.119048,0.142857,0.023810,0.000000,0.02381,Walmart,101.95,0.025964,0.093250,0,1,0,2010Q3,2010Q3,0.107158
3,0.000000,0.051282,0.333333,0.153846,0.000000,0.230769,0.000000,0.076923,0.128205,0.000000,0.025641,0.00000,Walmart,116.36,0.024115,0.294916,0,0,1,2010Q4,2010Q4,0.010682
4,0.000000,0.071429,0.309524,0.047619,0.000000,0.261905,0.023810,0.119048,0.119048,0.000000,0.023810,0.02381,Walmart,104.19,0.043883,1.264822,0,0,0,2011Q1,2011Q1,-0.046004


In [116]:
topic_stock = topic_stock.merge(sp, how="left", left_on=["quarter"], right_on=["quarter"])
topic_stock.shape

(189, 23)

In [118]:
# topic_stock = topic_stock.drop(["quarter_x", "quarter_y", "actual_quarter"], axis =1)
topic_stock.head()

,1,2,3,4,5,6,7,8,9,10,11,12,company,Revenue,yoy_growth,pci,Q2,Q3,Q4,actual_quarter,quarter,Price,sp_index_change
0,0.027778,0.083333,0.305556,0.000000,0.027778,0.194444,0.111111,0.083333,0.166667,0.000000,0.000000,0.00000,Walmart,99.81,0.059104,0.401415,0,0,0,2010Q1,2010Q1,0.025263,0.032163
1,0.000000,0.076923,0.256410,0.051282,0.051282,0.179487,0.025641,0.102564,0.179487,0.025641,0.051282,0.00000,Walmart,103.73,0.028251,0.475072,1,0,0,2010Q2,2010Q2,-0.133718,-0.125108
2,0.023810,0.095238,0.285714,0.095238,0.023810,0.166667,0.000000,0.119048,0.142857,0.023810,0.000000,0.02381,Walmart,101.95,0.025964,0.093250,0,1,0,2010Q3,2010Q3,0.107158,0.110797
3,0.000000,0.051282,0.333333,0.153846,0.000000,0.230769,0.000000,0.076923,0.128205,0.000000,0.025641,0.00000,Walmart,116.36,0.024115,0.294916,0,0,1,2010Q4,2010Q4,0.010682,0.097187
4,0.000000,0.071429,0.309524,0.047619,0.000000,0.261905,0.023810,0.119048,0.119048,0.000000,0.023810,0.02381,Walmart,104.19,0.043883,1.264822,0,0,0,2011Q1,2011Q1,-0.046004,0.042426


In [119]:
topic_stock.to_csv("topic_stock.csv", index="False")